# Identifikacija zvezde

Proces identifikacije zvezde poteka z uporabo <a href="https://unilj-my.sharepoint.com/:b:/g/personal/aleksander_grm_fpp_uni-lj_si/EQhyqtLl3lNKsl2Jo21S4f8Bidd2DuJ_hLI1jO1bR5Cg8w">Navtičnega almanaha</a> (*Dec* in *Sha* zvezde) in meritvijo višine $H$ ter azimuta $\omega$ zvezde.

Poleg same meritve moramo določiti še čas meritve in položaj meritve. Položaje vnašamo v obliki, kot je v navtiki dogovor 

$\varphi = \#\#^\circ \, \#\#.\#' \, \mathrm{N/S}$, $\lambda = \#\#\#^\circ \, \#\#.\#' \, \mathrm{E/W}$, 

kjer imamo stopinje kot celo število in minute kot število na 1 decimalno mesto natančno.

Format vnosa položaja:
- LAT-$\varphi$ in LONG-$\lambda$ vnašamo v formatu [stopinje, minute, stran neba]

Primer vnosa položaja:
- $\varphi$: [12, 34.5, 'N']
- $\lambda$: [117, 12.6, 'W']
- stran neba mora biti v **enojnih navednicah**, kjer so dovoljeni znaki 'N', 'S', 'E', 'W'

Čas meritve zapišemo v obliki
- datum: *date*=[day,month,year]
- ura: *time*=[hour,minutes, seconds]

Višino in azumut zapišemo
- višina: $h=\#\#^\circ \#\#'$=[deg,min]
- azimut: $\omega=\#\#\#^\circ$=[deg]

<hr>

**Postopek identifikacije zvezde**

Določimo vhode parametre:
1. Določimo naš položaj meritve $P_0$
2. Določimo čas meritve *date*, *time*
3. Izmerimo višino nebesnega telesa s sekstantom $H$
4. Izmerimo azimut do nebesnega telesa $\omega$

Nato pričnemo z izračunom, ki poteka v sledečih korakih:
1. pretvorimo izmerjene količine:
\begin{align*}
    Z & = 90^\circ - H \\
    \psi & = 90^\circ - \varphi
\end{align*}

2. izračunamo razdaljo $p$ in nato deklinacijo $\delta$ izmeerjene zvezde
\begin{align*}
    \cos p & = \cos Z \cos \psi + \sin Z \sin \psi \cos \omega\\
    \delta & = 90^\circ - p
\end{align*}

3. določimo mestni časovni kot zvezde $\mathrm{Lha}$
$$
\cos \mathrm{Lha} = \frac{\cos Z - \cos p \cos \psi}{\sin p \sin \psi}
$$
kjer imamo pogoj, če je $\omega < 180^\circ$ sledi, da je potrebno izračunani $\mathrm{Lha}$ popraviti in sicer sledi $\to$ $\mathrm{Lha} = 360^\circ - \mathrm{Lha}$

4. s pomočjo $\mathrm{Lha}$ lahko sedaj določimo še siderični časovni kot zvezde $\mathrm{Sha}$
\begin{align*}
    \mathrm{Lha}_{\gamma} & = \mathrm{Gha}_{\gamma} + (\pm \lambda)\\
    \mathrm{Sha} & = \mathrm{Lha} - \mathrm{Lha}_{\gamma}
\end{align*}
kjer imamo pogoj, če je $\mathrm{Sha} < 0^\circ$ sledi, da je potrebno izračunani $\mathrm{Sha}$ popraviti in sicer sledi $\to$ $\mathrm{Sha} = 360^\circ + \mathrm{Sha}$

Sedaj ko imamo določeno deklinacijo zvezde $\delta$ in njen siderični časovni kot $\mathrm{Sha}$ lahko s pomočjo *Navtičnega almanaha* identificiramo ime zvezde.

In [ ]:
import os, sys

# add custom modules and astro data path 
pp = '../nav_tools/'
sys.path.append(pp)

In [ ]:
import math as mat
import numpy as np
import matplotlib.pyplot as mpl
mpl.rcParams['text.usetex'] = True
mpl.rcParams.update({'font.size': 7})

import celestialdata as cdata
import navigationalstars as ns
import navtools as nt

In [ ]:
# poišče vse vidne zvezde nad obzorjem
def findAllVisibleStars(t,pos,ns_db):
    
    star_ids = ns_db.keys()
    h0 = 0
    h1 = 90
    
    vs = []
    for s_id in star_ids:
        sdd = cd.get_star_data(s_id,t)
        saz = cd.get_star_altaz(s_id,t,pos)
        if saz['alt'] > h0 and saz['alt'] < h1:
            s_name = ns_db[s_id][0]
            vs.append({'name':s_name, 'dec':sdd['dec'], 'sha':sdd['sha'], 'alt':saz['alt'], 'az':saz['az']})
    
    return vs

In [ ]:
# Find star dec and sha
def getStarDecAndSha(s_h,s_w,pos,date,time):
    
    t = [date[0],date[1],date[2],time[0],time[1],time[2]]
    msg = ''
 
    gha_a = cd.get_aries_gha(t)
    lha_a = gha_a + pos[1]

    s_z = nt.deg2rad(90.0 - s_h)
    s_psi = nt.deg2rad(90.0 - pos[0])

    # find declination
    cos_s_p = mat.cos(s_z)*mat.cos(s_psi) + mat.sin(s_z)*mat.cos(s_psi)*mat.cos(nt.deg2rad(s_w))
    if mat.fabs(cos_s_p) > 1.0: msg='cos_p > 1.0 ({:.5f})'.format(cos_s_p)
    if cos_s_p > 1.0:
        cos_s_p = 1.0
    if cos_s_p < -1.0:
        cos_s_p = -1.0
    s_p = mat.acos(cos_s_p) 
    dec = 90.0 - nt.rad2deg(s_p)

    # find local hour angle
    # !! due to measured data it can be out of range (out of interval [-1,1]) !!!
    cos_lha = (mat.cos(s_z) - mat.cos(s_p)*mat.cos(s_psi))/(mat.sin(s_p)*mat.sin(s_psi))
    if mat.fabs(cos_lha) > 1.0: 
        if msg == '':
            msg='cos_lha > 1.0 ({:.5f})'.format(cos_lha)
        else:
            msg = msg + ' and cos_lha > 1.0 ({:.5f})'.format(cos_lha)
    if cos_lha > 1.0:
        cos_lha = 1.0
    if cos_lha < -1.0:
        cos_lha = -1.0
        
    lha = nt.rad2deg(mat.acos(cos_lha))
    if s_w < 180.0:
        lha = 360 - lha

    # find siderial hour angle
    sha = lha - lha_a
    if sha < 0:
        sha = 360 + sha
        
    return [dec,sha,msg]

In [ ]:
# poišči ime zvezde na podalgi izračunane deklinacije in sha
def findStarName(ns_db,pos,date,time,dec,sha):
    
    t = [date[0],date[1],date[2],time[0],time[1],time[2]]
    
    vs = findAllVisibleStars(t,pos,ns_db)
    
    dist = []
    stars = []
    for s in vs:
        dd = s['dec'] - dec
        ds = s['sha'] - sha
        da = mat.fabs(dd) + mat.fabs(ds)
        dist.append(da)
        stars.append(s)

        #print('name: {:12s}, dist: {:6.2f}, dec: {:5.2f}, sha: {:5.2f}'.format(
        #    s['name'],da,s['dec'],s['sha']))

    # find the star with minimal dec+sha distance
    min_dist = min(dist)
    min_idx = dist.index(min_dist)
    ac = stars[min_idx]
    
    return ac

In [ ]:
# load Astro database
cd = cdata.CelestialData(pp)
ns_db = cd.get_nav_stars_db()

In [ ]:
# *** inputs ***
date = [2022,2,21] # [yyyy, mm, dd]
time = [19,0,0]    # [HH,MM,SS] in UTC

# Apparent position
fi = [46,6.08,'N']  # Latitude
la = [13,38.85,'E'] # Longitude
h = 20.0 # observer height in [m]

# Star measurements
s_h = [51, 10] # observed height [deg, min.dec]
s_w = 182.0    # observed azimuth in [deg]


pos = [nt.nav2dd(fi), nt.nav2dd(la), h]
s_h = nt.dms2dd(s_h)

print('MEASURED data:')
print('  -> alt: {:s}'.format(nt.prettyPrintAlt(s_h)))
print('  ->  az: {:s}'.format(nt.prettyPrintAz(s_w)))

print()
[dec, sha, msg] = getStarDecAndSha(s_h, s_w, pos, date, time)
if msg == '':
    print('CALCULATED data:')
else:
    print('CALCULATED data (*WARNING* {:s}):'.format(msg))
print('  -> dec:  {:s}'.format(nt.prettyPrintDec(dec)))
print('  -> sha: {:s}'.format(nt.prettyPrintHA(sha)))
    
ss = findStarName(ns_db,pos,date,time,dec,sha) 

print()
print('Identification with cloasest star from ALMANACH:')
print('  -> name: {:s}'.format(ss['name'].upper()))
print('  ->  dec:  {:s}'.format(nt.prettyPrintDec(ss['dec'])))
print('  ->  sha: {:s}'.format(nt.prettyPrintHA(ss['sha'])))
print('  ->  alt:  {:s}'.format(nt.prettyPrintAlt(ss['alt'])))
print('  ->   az: {:s}'.format(nt.prettyPrintAz(ss['az'])))
